In [0]:
df = spark.read.csv('/mnt/sdslake/sds/lake/workspaces/cloudiq/test/test/ratings.csv', header=True,inferSchema=True)
df.limit(15).toPandas()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Create a Spark session
spark = SparkSession.builder.appName("AmazonProductRecommendation").getOrCreate()


df = spark.read.csv('/mnt/sdslake/sds/lake/workspaces/cloudiq/test/test/ratings.csv', header=True,inferSchema=True)

# Prepare the data (assuming you have columns: 'user', 'item', 'rating')
df = df.withColumnRenamed('userId', 'user').withColumnRenamed('movieId', 'item').withColumnRenamed('rating', 'rating')

# Create an ALS model
als = ALS(rank=10, maxIter=10, regParam=0.1, userCol='user', itemCol='item', ratingCol='rating')
model = als.fit(df)

# Evaluate the model
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
predictions = model.transform(df)
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Make recommendations for users (e.g., user 0)
user_recs = model.recommendForAllUsers(5)
user_0_recs = user_recs.where(user_recs.user == 24).select("recommendations.item", "recommendations.rating").collect()
print(f"Recommendations for user 24: {user_0_recs}")

Root Mean Squared Error (RMSE): 0.5933279277739512
Recommendations for user 24: [Row(item=[33649, 3379, 171495, 74282, 7121], rating=[4.727833271026611, 4.6474504470825195, 4.5525336265563965, 4.409307479858398, 4.388001441955566])]
